# Install required libraries

In [1]:
import pandas as pd
import requests as req
import numpy as np
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
!pip install geopy
from geopy.geocoders import Nominatim



     |████████████████████████████████| 94 kB 5.6 MB/s  eta 0:00:01


# Create pandas dataframe of the neighborhoods and associated latitudes and longitudes

In [27]:
URL = 'https://en.wikipedia.org/wiki/Neighborhoods_in_New_Orleans'
r= req.get(URL)
dfs = pd.read_html(URL)
df = dfs[0]
print(len(df.Neighborhood.unique()))
df.head()

72


,Neighborhood,Longitude,Latitude
0,U.S. NAVAL BASE,-90.026093,29.946085
1,ALGIERS POINT,-90.051606,29.952462
2,WHITNEY,-90.042357,29.947200
3,AUDUBON,-90.121450,29.932994
4,OLD AURORA,-90.000000,29.924440


# Define the latitude and longitude of New Orleans and create associated map

In [3]:
address = 'New Orleans'

geolocator = Nominatim(user_agent="geomap")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of New Orleans are 29.9499323, -90.0701156.


In [4]:
# create map of New Orleans using latitude and longitude values
map_no = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_no)  
    
map_no



# Using Foursquare get the top 100 venues in each neighbourhood in New Orleans, within a radius of 500 meters

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = req.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
#run the function to get the nearby venues, this will show the names of each neighbourhood as it is executed
no_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

U.S. NAVAL BASE
ALGIERS POINT
WHITNEY
AUDUBON
OLD AURORA
B. W. COOPER
BAYOU ST. JOHN
BEHRMAN
BLACK PEARL
BROADMOOR
MARLYVILLE - FONTAINEBLEAU
GERT TOWN
MID-CITY
ST. CLAUDE
CENTRAL BUSINESS DISTRICT
FRENCH QUARTER
CENTRAL CITY
LAKE CATHERINE
VILLAGE DE LEST
VIAVANT - VENETIAN ISLES
NEW AURORA - ENGLISH TURN
TALL TIMBERS - BRECHTEL
FISCHER DEV
McDONOGH
LOWER GARDEN DISTRICT
ST. THOMAS DEV
EAST RIVERSIDE
IRISH CHANNEL
TOURO
MILAN
UPTOWN
WEST RIVERSIDE
EAST CARROLLTON
FRERET
GARDEN DISTRICT
LEONIDAS
HOLLYGROVE
TULANE - GRAVIER
TREME - LAFITTE
SEVENTH WARD
MARIGNY
ST. ROCH
DIXON
LAKEWOOD
NAVARRE
CITY PARK
LAKEVIEW
WEST END
LAKESHORE - LAKE VISTA
FILMORE
ST. BERNARD AREA
DILLARD
ST. ANTHONY
LAKE TERRACE & OAKS
MILNEBURG
PONTCHARTRAIN PARK
GENTILLY WOODS
GENTILLY TERRACE
DESIRE AREA
FLORIDA AREA
FLORIDA DEV
LOWER NINTH WARD
BYWATER
HOLY CROSS
PINES VILLAGE
PLUM ORCHARD
READ BLVD WEST
READ BLVD EAST
WEST LAKE FOREST
LITTLE WOODS
FAIRGROUNDS
IBERVILLE


In [8]:
print(len(no_venues.Neighborhood.unique()))
print(len(df['Neighborhood']))
print(no_venues.shape)
no_venues.head()

67
72
(924, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,U.S. NAVAL BASE,29.946085,-90.026093,The Mighty Missisippi,29.949695,-90.023710,Boat or Ferry
1,ALGIERS POINT,29.952462,-90.051606,Tout de Suite Café,29.952121,-90.051090,Café
2,ALGIERS POINT,29.952462,-90.051606,Congregation Coffee Roasters,29.951918,-90.053395,Coffee Shop
3,ALGIERS POINT,29.952462,-90.051606,The Crown & Anchor,29.951416,-90.054220,Bar
4,ALGIERS POINT,29.952462,-90.051606,Levee @ Algiers Point,29.951760,-90.048747,Scenic Lookout


In [9]:
#It appears that some of the original neghborhoods (72) did not get included in the determination of no_venues ((66))
#Reduce df to include only the neighborhoods for which venue categories and venues were determined.

df = df[df['Neighborhood'].isin(no_venues.Neighborhood)]
print(len(df['Neighborhood']))

67


## Apply onehotencoding to 'Venue Category' to show how many of each Venue Category is in each neighborhood

In [10]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

In [11]:

no_onehot = pd.get_dummies(no_venues[['Venue Category']], prefix="", prefix_sep="")
# add the neighborhood column to dataframe
no_onehot['Neighborhood'] = no_venues['Neighborhood'] 

# move neighborhood column to the first column
no_onehot = no_onehot[ ['Neighborhood'] + [ col for col in no_onehot.columns if col != 'Neighborhood' ] ]


no_onehot.head()



,Neighborhood,ATM,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Vehicle Inspection Station,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,U.S. NAVAL BASE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALGIERS POINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALGIERS POINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALGIERS POINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALGIERS POINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
no_grouped = no_onehot.groupby('Neighborhood').mean().reset_index()
no_grouped

,Neighborhood,ATM,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Vehicle Inspection Station,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,ALGIERS POINT,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.047619,0.0,0.000000
1,AUDUBON,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,B. W. COOPER,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
3,BAYOU ST. JOHN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
4,BLACK PEARL,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,UPTOWN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.000000,0.0,0.037037
63,WEST END,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
64,WEST LAKE FOREST,0.0,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
65,WEST RIVERSIDE,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.037037


In [13]:
num_top_venues = 5

for hood in no_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = no_grouped[no_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALGIERS POINT----
           venue  freq
0  Boat or Ferry  0.14
1            Bar  0.14
2           Café  0.05
3  Grocery Store  0.05
4    Pizza Place  0.05


----AUDUBON----
            venue  freq
0            Park  0.17
1  Scenic Lookout  0.08
2  Sandwich Place  0.08
3           Plaza  0.08
4   Smoothie Shop  0.08


----B. W. COOPER----
                  venue  freq
0            Boxing Gym  0.17
1     Recreation Center  0.17
2            Food Truck  0.17
3         Moving Target  0.17
4  Gym / Fitness Center  0.17


----BAYOU ST. JOHN----
                    venue  freq
0             Gas Station  0.11
1  Furniture / Home Store  0.11
2       Convenience Store  0.11
3    Other Great Outdoors  0.11
4          Sandwich Place  0.11


----BLACK PEARL----
            venue  freq
0      Food Truck  0.25
1   Grocery Store  0.25
2  Farmers Market  0.25
3           Plaza  0.25
4             ATM  0.00


----BROADMOOR----
               venue  freq
0  Recreation Center  0.17
1        Music Ven

In [14]:
#put the above into a sorted Pandas dataframe:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = no_grouped['Neighborhood']

for ind in np.arange(no_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(no_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALGIERS POINT,Bar,Boat or Ferry,Coffee Shop,New American Restaurant,Café,Park,Gift Shop,Scenic Lookout,Grocery Store,Music Venue
1,AUDUBON,Park,Light Rail Station,College Arts Building,Outdoors & Recreation,Scenic Lookout,Sandwich Place,Smoothie Shop,Coffee Shop,Playground,Plaza
2,B. W. COOPER,Recreation Center,Gym / Fitness Center,Food Truck,Moving Target,Home Service,Boxing Gym,French Restaurant,Food Service,Food Court,Food & Drink Shop
3,BAYOU ST. JOHN,Convenience Store,Sandwich Place,Furniture / Home Store,Gas Station,Other Great Outdoors,Park,Playground,Lounge,Home Service,Donut Shop
4,BLACK PEARL,Farmers Market,Food Truck,Grocery Store,Plaza,Fast Food Restaurant,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop


## Cluster the Neighborhoods in New Orleans

In [15]:
#run K-means to cluster the neighborhoods.  Start with 7 clusters
kclusters = 7

no_grouped_clustering = no_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(no_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([6, 6, 6, 6, 5, 6, 6, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 1, 6, 2, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 2, 5, 0, 6, 3, 5, 2, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6,
       6], dtype=int32)

In [16]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

no_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
no_merged = no_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

no_merged.head() # check the last columns!

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,U.S. NAVAL BASE,-90.026093,29.946085,6,Boat or Ferry,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
1,ALGIERS POINT,-90.051606,29.952462,6,Bar,Boat or Ferry,Coffee Shop,New American Restaurant,Café,Park,Gift Shop,Scenic Lookout,Grocery Store,Music Venue
2,WHITNEY,-90.042357,29.947200,6,Nightclub,Dessert Shop,Yoga Studio,Financial or Legal Service,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
3,AUDUBON,-90.121450,29.932994,6,Park,Light Rail Station,College Arts Building,Outdoors & Recreation,Scenic Lookout,Sandwich Place,Smoothie Shop,Coffee Shop,Playground,Plaza
4,OLD AURORA,-90.000000,29.924440,6,Nightlife Spot,Playground,Gym / Fitness Center,Yoga Studio,Financial or Legal Service,Food Truck,Food Service,Food Court,Food & Drink Shop,Food


In [17]:
#visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(no_merged['Latitude'], no_merged['Longitude'], no_merged['Neighborhood'], no_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [18]:
no_merged.head()

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,U.S. NAVAL BASE,-90.026093,29.946085,6,Boat or Ferry,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
1,ALGIERS POINT,-90.051606,29.952462,6,Bar,Boat or Ferry,Coffee Shop,New American Restaurant,Café,Park,Gift Shop,Scenic Lookout,Grocery Store,Music Venue
2,WHITNEY,-90.042357,29.947200,6,Nightclub,Dessert Shop,Yoga Studio,Financial or Legal Service,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
3,AUDUBON,-90.121450,29.932994,6,Park,Light Rail Station,College Arts Building,Outdoors & Recreation,Scenic Lookout,Sandwich Place,Smoothie Shop,Coffee Shop,Playground,Plaza
4,OLD AURORA,-90.000000,29.924440,6,Nightlife Spot,Playground,Gym / Fitness Center,Yoga Studio,Financial or Legal Service,Food Truck,Food Service,Food Court,Food & Drink Shop,Food


In [19]:
#Examine Clusters - Cluster 1

no_merged.loc[no_merged['Cluster Labels'] == 0, no_merged.columns[[0] + list(range(4, no_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,READ BLVD WEST,Gas Station,Yoga Studio,Furniture / Home Store,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop


In [20]:
#Examine Clusters - Cluster 2

no_merged.loc[no_merged['Cluster Labels'] == 1, no_merged.columns[[0] + list(range(4, no_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,LAKE CATHERINE,Clothing Store,Yoga Studio,Financial or Legal Service,Fried Chicken Joint,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food


In [21]:
#Examine Clusters - Cluster 3

no_merged.loc[no_merged['Cluster Labels'] == 2, no_merged.columns[[0] + list(range(4, no_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,ST. CLAUDE,Plaza,Park,Yoga Studio,Financial or Legal Service,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
16,CENTRAL CITY,Park,Grocery Store,Cajun / Creole Restaurant,Seafood Restaurant,Fried Chicken Joint,Yoga Studio,Fishing Store,Food Truck,Food Service,Food Court
48,LAKESHORE - LAKE VISTA,Harbor / Marina,Food,Park,Fried Chicken Joint,Food Truck,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market
55,PONTCHARTRAIN PARK,Park,Yoga Studio,Insurance Office,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop


In [22]:
#Examine Clusters - Cluster 4

no_merged.loc[no_merged['Cluster Labels'] == 3, no_merged.columns[[0] + list(range(4, no_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,WEST END,Historic Site,Playground,Yoga Studio,Farmers Market,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop
52,ST. ANTHONY,Playground,Trail,Yoga Studio,Fast Food Restaurant,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food


In [23]:
#Examine Clusters - Cluster 5

no_merged.loc[no_merged['Cluster Labels'] == 4, no_merged.columns[[0] + list(range(4, no_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,FLORIDA AREA,Wings Joint,Dessert Shop,BBQ Joint,Seafood Restaurant,Yoga Studio,Fast Food Restaurant,French Restaurant,Food Truck,Food Service,Food Court
60,FLORIDA DEV,Wings Joint,Yoga Studio,Fast Food Restaurant,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop


In [24]:
#Examine Clusters - Cluster 6

no_merged.loc[no_merged['Cluster Labels'] == 5, no_merged.columns[[0] + list(range(4, no_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,BLACK PEARL,Farmers Market,Food Truck,Grocery Store,Plaza,Fast Food Restaurant,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop
50,ST. BERNARD AREA,Rental Car Location,Dessert Shop,Gym,Fast Food Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop
67,READ BLVD EAST,Pharmacy,Insurance Office,Scenic Lookout,Bank,Yoga Studio,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
69,LITTLE WOODS,Pharmacy,Seafood Restaurant,Yoga Studio,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market


In [25]:
#Examine Clusters - Cluster 7

no_merged.loc[no_merged['Cluster Labels'] == 6, no_merged.columns[[0] + list(range(4, no_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,U.S. NAVAL BASE,Boat or Ferry,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
1,ALGIERS POINT,Bar,Boat or Ferry,Coffee Shop,New American Restaurant,Café,Park,Gift Shop,Scenic Lookout,Grocery Store,Music Venue
2,WHITNEY,Nightclub,Dessert Shop,Yoga Studio,Financial or Legal Service,French Restaurant,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
3,AUDUBON,Park,Light Rail Station,College Arts Building,Outdoors & Recreation,Scenic Lookout,Sandwich Place,Smoothie Shop,Coffee Shop,Playground,Plaza
4,OLD AURORA,Nightlife Spot,Playground,Gym / Fitness Center,Yoga Studio,Financial or Legal Service,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
5,B. W. COOPER,Recreation Center,Gym / Fitness Center,Food Truck,Moving Target,Home Service,Boxing Gym,French Restaurant,Food Service,Food Court,Food & Drink Shop
6,BAYOU ST. JOHN,Convenience Store,Sandwich Place,Furniture / Home Store,Gas Station,Other Great Outdoors,Park,Playground,Lounge,Home Service,Donut Shop
9,BROADMOOR,Recreation Center,Food Truck,Taco Place,Coffee Shop,Trail,Music Venue,Fast Food Restaurant,Food Service,Food Court,Food & Drink Shop
10,MARLYVILLE - FONTAINEBLEAU,Bar,Nightclub,Movie Theater,Food & Drink Shop,Athletics & Sports,Shopping Mall,Campground,Lounge,Yoga Studio,Food Truck
11,GERT TOWN,Rental Service,Seafood Restaurant,Asian Restaurant,Tennis Court,Financial or Legal Service,Bakery,Food Truck,Food Service,Food Court,Food & Drink Shop
